<a href="https://colab.research.google.com/github/fansie1/chemRxnClassification/blob/dev_trainer/chemRxnBetter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install datasets
!pip install accelerate
!pip install evaluate
!pip install wandb
!pip install sacremoses
!pip install spacy
!pip install -U --no-cache-dir gdown --pre
!pip install seqeval

import pdb
import wandb
use_wandb = False
if use_wandb:
  wandb.login()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 60.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 77.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 77.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.5/189.5 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!gdown --no-cookies https://drive.google.com/uc?id=1--UFHJY3ldytNTsLcseIMDXEyzFeYEuj
!gdown --no-cookies https://drive.google.com/uc?id=1-oYW7JgHyhFj1HpbYvpHdsK8y-RwF2pW
!gdown --no-cookies https://drive.google.com/uc?id=1-KkDBrlyr3DF91BOTgp-vgCLwMe2JAmX

Downloading...
From: https://drive.google.com/uc?id=1--UFHJY3ldytNTsLcseIMDXEyzFeYEuj
To: /content/train_optim_concat.txt
100% 1.57M/1.57M [00:00<00:00, 167MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-oYW7JgHyhFj1HpbYvpHdsK8y-RwF2pW
To: /content/test_optim_concat.txt
100% 167k/167k [00:00<00:00, 111MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-KkDBrlyr3DF91BOTgp-vgCLwMe2JAmX
To: /content/dev_optim_concat.txt
100% 162k/162k [00:00<00:00, 115MB/s]


In [3]:
# 获取数据集
# train = "/content/drive/MyDrive/chemrxnconfig/prod/train_optim_concat.txt"
# dev = "/content/drive/MyDrive/chemrxnconfig/prod/dev_optim_concat.txt"
# test = "/content/drive/MyDrive/chemrxnconfig/prod/test_optim_concat.txt"
train = "/content/train_optim_concat.txt"
dev = "/content/dev_optim_concat.txt"
test = "/content/test_optim_concat.txt"

bert_tag_label_id = -100
spacy_tag_label_id = -100
pad_label_id = -100
pad_token = 0
prod_labels = ["O", "B-Prod", "I-Prod"]

label2id = {context: i for i, context in enumerate(prod_labels)}
id2label = {i: context for i, context in enumerate(prod_labels)}
def read_examples_from_file(file_path):
    guid_index = 1
    examples = []
    with open(file_path, encoding="utf-8") as f:
        words, labels = [], []
        metainfo = None
        for line in f:
            line = line.rstrip()
            if line.startswith("#\tpassage"):
                metainfo = line
            elif line == "":
                if words:
                    examples.append({
                            "guid":f"{guid_index}",
                            "words":words,
                            "metainfo":metainfo,
                            "labels":labels
                        })
                    guid_index += 1
                    words, labels = [], []
            else:
                splits = line.split("\t")
                words.append(splits[0])
                if len(splits) > 1:
                    labels.append(splits[-1])
                else:
                    # Examples could have no label for plain test files
                    labels.append("O")
        if words:

            examples.append({
                    "guid":f"{guid_index}",
                    "words":words,
                    "metainfo":metainfo,
                    "labels":labels
                 })

    return examples

train_input = read_examples_from_file(train)
dev_input = read_examples_from_file(dev)
test_input = read_examples_from_file(test)

In [4]:
# tokenize
from transformers import AutoTokenizer, AutoConfig
import spacy
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
spacy_model = spacy.load("en_core_web_sm")
en = English()

checkpoint = "jiangg/chembert_cased"
# checkpoint = "dmis-lab/biobert-large-cased-v1.1-squad"
# checkpoint = "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext"
# checkpoint = "microsoft/biogpt"
# spacy_pos_token = {
#     "ADJ": "[unused1]",
#     "ADP": "[unused2]",
#     "ADV": "[unused3]",
#     "AUX": "[unused4]",
#     "CCONJ": "[unused5]",
#     "DET": "[unused6]",
#     "INTJ": "[unused7]",
#     "NOUN": "[unused8]",
#     "NUM": "[unused9]",
#     "PART": "[unused10]",
#     "PRON": "[unused11]",
#     "PROPN": "[unused12]",
#     "PUNCT": "[unused13]",
#     "SCONJ": "[unused14]",
#     "SYM": "[unused15]",
#     "VERB": "[unused16]",
#     "X": "[unused17]",
# }

# just useing the  noun and verb
spacy_pos_token = {"NOUN": "[unused8]", "VERB": "[unused16]",}

CLS = "[CLS]"
SEP = "[SEP]"
max_sentence = 256
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

token_cache_path = {
    "train": "token_cache_train",
    "validation": "token_cache_validation",
    "test": "token_cache_test"
}

output_dir = "/content"


def build_spacy(sent_list):
  return {
      "spacy": spacy_model(' '.join(sent_list)),
      "find_idx": build_sent_idx(sent_list)
}


def build_sent_idx(sent_list):
  total_len = 0
  find_idx = []
  for i, word in enumerate(sent_list):
    find_idx.append(str(total_len) + "-" + str(total_len + len(word) - 1))
    # stridx2listidx[str(total_len) + "-" + str(total_len + len(word))] = i
    total_len = total_len + len(word) + 1
  return find_idx

def do_find_idx(find_idx, idx):
  for i, func in enumerate(find_idx):
    func_list = func.split("-")
    if (int(func_list[0]) <= idx and int(func_list[1]) >= idx):
      return i


def check_append_word(tokens_list, word):
  if (len(tokens_list) + 1) > max_sentence - 1:
    return
  tokens_list.append(word)

In [5]:
# dataloader
from datasets import Dataset, DatasetDict


def tokenize_test(example):
  tokens = []
  label_ids = []
  label_map = label2id
  pad_token_label_id = -100
  max_seq_length = max_sentence
  sep_token = SEP
  cls_token = CLS
  sequence_a_segment_id = 0
  mask_padding_with_zero = True
  cls_token_segment_id = 0
  pad_token_segment_id = 0
  for word, label in zip(example["words"], example["labels"]):
      word_tokens = tokenizer.tokenize(word)
      # word_tokens = word_tokens[:5]

      if len(word_tokens) > 0:
          tokens.extend(word_tokens)
          label_ids.extend([label_map[label]] + [pad_token_label_id] * (len(word_tokens) - 1))

  if len(tokens) > max_seq_length - 2:
      tokens = tokens[: (max_seq_length - 2)]
      label_ids = label_ids[: (max_seq_length - 2)]

  tokens += [sep_token]
  label_ids += [pad_token_label_id]
  segment_ids = [sequence_a_segment_id] * len(tokens)

  tokens = [cls_token] + tokens
  label_ids = [pad_token_label_id] + label_ids
  segment_ids = [cls_token_segment_id] + segment_ids

  input_ids = tokenizer.convert_tokens_to_ids(tokens)

  # The mask has 1 for real tokens and 0 for padding tokens. Only real
  # tokens are attended to.
  input_mask = [1 if mask_padding_with_zero else 0] * len(input_ids)

  # Zero-pad up to the sequence length.
  seq_length = len(input_ids)
  padding_length = max_seq_length - len(input_ids)
  input_ids += [pad_token] * padding_length
  input_mask += [0 if mask_padding_with_zero else 1] * padding_length
  segment_ids += [pad_token_segment_id] * padding_length
  label_ids += [pad_token_label_id] * padding_length

  decoder_mask = [(x != pad_token_label_id) for x in label_ids]
  return {
  "input_ids": input_ids,
  "token_type_ids": segment_ids,
  "attention_mask": input_mask,
  "labels": label_ids,
  "decoder_mask": decoder_mask
}

def tokenize_function(example):
  word_list = example["words"]
  original_labels = example["labels"]
  spacy_result = build_spacy(word_list)
  spacy = spacy_result["spacy"]
  sent_idx = spacy_result["find_idx"]
  tokens = [CLS]
  labels = [bert_tag_label_id]
  last_idx = 0
  sub_token_tag = None
  for _, word in enumerate(spacy):
    original_idx = do_find_idx(sent_idx, word.idx)
    # the sub token from same token
    if original_idx == last_idx:
      if not sub_token_tag and spacy_pos_token.get(word.pos_):
        sub_token_tag = spacy_pos_token.get(word.pos_)
    # the sub token from new token
    else:
      last_token = word_list[last_idx]
      first_sub_token = True
      for sub_token in tokenizer.tokenize(last_token):
        check_append_word(tokens, sub_token)
        label_tag = original_labels[last_idx] if first_sub_token else "O"
        check_append_word(labels,  label2id[label_tag])
        first_sub_token = False
      # add verb or noun token
      if sub_token_tag:
        check_append_word(tokens, sub_token_tag)
        check_append_word(labels, spacy_tag_label_id)
      last_idx = original_idx
  assert len(tokens) == len(labels)
  input_ids, token_type_ids, attention_mask, labels = padding_fn(
      tokenizer.convert_tokens_to_ids(tokens),
      [0] * len(tokens),
      [1] * len(tokens),
      labels,
  )
  decoder_mask = [(x != pad_label_id) for x in labels]
  assert len(input_ids) == max_sentence
  assert len(token_type_ids) == max_sentence
  assert len(attention_mask) == max_sentence
  assert len(labels) == max_sentence
  assert len(decoder_mask) == max_sentence
  return {
    "input_ids": input_ids,
    "token_type_ids": token_type_ids,
    "attention_mask": attention_mask,
    "labels": labels,
    "decoder_mask": decoder_mask
  }


def tokenize_function_no_spacy(example):
  tokens = []
  label_ids = []
  word_list = example["words"]
  original_labels = example["labels"]

  for word, label in zip(word_list, original_labels):
    word_tokens = tokenizer.tokenize(word)
    if len(word_tokens) > 0:
      tokens.extend(word_tokens)
      label_ids.extend([label2id[label]] + [pad_label_id] * (len(word_tokens) - 1))

  if len(tokens) > max_sentence - 2:
    tokens = tokens[:(max_sentence - 2)]
    label_ids = label_ids[:(max_sentence - 2)]

  tokens += [SEP]
  label_ids += [pad_label_id]

  tokens = [CLS] + tokens
  label_ids = [pad_label_id] + label_ids

  input_ids, token_type_ids, attention_mask, labels = padding_fn(
      tokenizer.convert_tokens_to_ids(tokens),
      [0] * len(tokens),
      [1] * len(tokens),
      label_ids,
  )
  decoder_mask = [(x != pad_label_id) for x in label_ids]
  assert len(input_ids) == max_sentence
  assert len(token_type_ids) == max_sentence
  assert len(attention_mask) == max_sentence
  assert len(label_ids) == max_sentence
  assert len(decoder_mask) == max_sentence
  return {
    "input_ids": input_ids,
    "token_type_ids": token_type_ids,
    "attention_mask": attention_mask,
    "labels": label_ids,
    "decoder_mask": decoder_mask
  }

def padding_fn(input_ids, token_type_ids, attention_mask, labels):
  pad_len = max_sentence - len(input_ids)
  input_ids += [pad_token] * pad_len
  token_type_ids += [0] * pad_len
  attention_mask += [0] * pad_len
  labels += [pad_label_id] * pad_len
  return input_ids, token_type_ids, attention_mask, labels


config = AutoConfig.from_pretrained(
      checkpoint,
      num_labels=3,
      id2label=id2label,
      label2id=label2id,
)
hidden_size=768
num_train_epochs=1

train_dataset = Dataset.from_list(train_input)
dev_dataset = Dataset.from_list(dev_input)
test_dataset = Dataset.from_list(test_input)

datasetDict = DatasetDict({
    "train": train_dataset,
    "validation": dev_dataset,
    "test": test_dataset
})


tokenized_datasets = datasetDict.map(function=tokenize_test,
                      cache_file_names=token_cache_path,
                      load_from_cache_file=True)


tokenized_datasets = tokenized_datasets.remove_columns(["words", "guid", "metainfo"])
tokenized_datasets.set_format("torch")

Map:   0%|          | 0/6163 [00:00<?, ? examples/s]

Map:   0%|          | 0/698 [00:00<?, ? examples/s]

Map:   0%|          | 0/723 [00:00<?, ? examples/s]

In [6]:
# from transformers import default_data_collator
# from torch.utils.data import DataLoader
# batch_size = 16
# # data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer, return_tensors="pt")
# train_dataloader = DataLoader(tokenized_datasets["train"], shuffle=True,  batch_size=batch_size, collate_fn=default_data_collator)
# eval_dataloader = DataLoader(tokenized_datasets["validation"], batch_size=batch_size, collate_fn=default_data_collator)
# test_dataloader = DataLoader(tokenized_datasets["test"], batch_size=batch_size, collate_fn=default_data_collator)

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

class FocalLossSelf(nn.Module):
    r"""
        This criterion is a implemenation of Focal Loss, which is proposed in
        Focal Loss for Dense Object Detection.
            Loss(x, class) = - \alpha (1-softmax(x)[class])^gamma \log(softmax(x)[class])
        The losses are averaged across observations for each minibatch.
        Args:
            alpha(1D Tensor, Variable) : the scalar factor for this criterion
            gamma(float, double) : gamma > 0; reduces the relative loss for well-classiﬁed examples (p > .5),
                                   putting more focus on hard, misclassiﬁed examples
            size_average(bool): By default, the losses are averaged over observations for each minibatch.
                                However, if the field size_average is set to False, the losses are
                                instead summed for each minibatch.
    """
    def __init__(self, class_num, alpha=None, gamma=2, size_average=True):
        super(FocalLoss, self).__init__()
        if alpha is None:
            self.alpha = Variable(torch.ones(class_num, 1))
        else:
            if isinstance(alpha, Variable):
                self.alpha = alpha
            else:
                self.alpha = Variable(alpha)
        self.gamma = gamma
        self.class_num = class_num
        self.size_average = size_average

    def forward(self, inputs, targets):
        N = inputs.size(0)
        C = inputs.size(1)
        P = F.softmax(inputs)

        class_mask = inputs.data.new(N, C).fill_(0)
        class_mask = Variable(class_mask)
        ids = targets.view(-1, 1)
        class_mask.scatter_(1, ids.data, 1.)
        #print(class_mask)


        if inputs.is_cuda and not self.alpha.is_cuda:
            self.alpha = self.alpha.cuda()
        alpha = self.alpha[ids.data.view(-1)]

        probs = (P*class_mask).sum(1).view(-1,1)

        log_p = probs.log()
        #print('probs size= {}'.format(probs.size()))
        #print(probs)

        batch_loss = -alpha*(torch.pow((1-probs), self.gamma))*log_p
        #print('-----bacth_loss------')
        #print(batch_loss)


        if self.size_average:
            loss = batch_loss.mean()
        else:
            loss = batch_loss.sum()
        return loss

class FocalLoss(nn.CrossEntropyLoss):
    ''' Focal loss for classification tasks on imbalanced datasets '''

    def __init__(self, gamma=2, alpha=None, ignore_index=-100):
        super().__init__(weight=alpha, ignore_index=ignore_index)
        self.gamma = gamma

    def forward(self, input_, target):
        cross_entropy = super().forward(input_, target)
        # Temporarily mask out ignore index to '0' for valid gather-indices input.
        # This won't contribute final loss as the cross_entropy contribution
        # for these would be zero.
        target = target * (target != self.ignore_index).long()
        input_prob = torch.gather(F.softmax(input_, 1), 1, target.unsqueeze(1))
        loss = torch.pow(1 - input_prob, self.gamma) * cross_entropy
        return torch.mean(loss)

In [8]:
import torch
from torch import nn
from torch.nn import CrossEntropyLoss
from transformers import BertForTokenClassification, TrainingArguments, AutoConfig



device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
batch_size = 16
class BertForTagging(BertForTokenClassification):
    def __init__(self, config):
        super(BertForTagging, self).__init__(config)
        # self.classifier = nn.Linear(hidden_size, config.num_labels)
        self.init_weights()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        labels=None,
        decoder_mask=None,
    ):
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        sequence_output = outputs[0]
        sequence_output = self.dropout(sequence_output)
        logits = self.classifier(sequence_output)
        outputs = (logits,)

        loss_fct = CrossEntropyLoss()
        # loss_fct = FocalLoss(alpha = torch.tensor([0.001, 0.4995, 0.4995], device=device))
        active_loss = attention_mask.view(-1) == 1
        active_logits = logits.view(-1, self.num_labels)
        active_labels = torch.where(
            active_loss, labels.view(-1), torch.tensor(loss_fct.ignore_index).type_as(labels)
        )
        loss = loss_fct(active_logits, active_labels)
        outputs = (loss,) + outputs

        return outputs  # (loss), scores, (hidden_states), (attentions)

    def decode(self, logits, mask):
        preds = torch.argmax(logits, dim=2).cpu().numpy()
        batch_size, seq_len = preds.shape
        preds_list = [[] for _ in range(batch_size)]
        for i in range(batch_size):
            for j in range(seq_len):
                if mask[i, j]:
                    preds_list[i].append(preds[i,j])
        return preds_list

    def decode_labels(self, labels, mask):
        labels_np = labels.cpu().numpy()
        batch_size, seq_len = labels_np.shape
        labels_list = [[] for _ in range(batch_size)]
        for i in range(batch_size):
          for j in range(seq_len):
            if mask[i, j]:
              labels_list[i].append(labels_np[i, j])
        labels_list_test = [labels[i, mask[i]].tolist() for i in range(labels.shape[0])]
        return labels_list



In [9]:
from seqeval.metrics import f1_score as f1, precision_score as precision, recall_score as recall
from tqdm import tqdm
import numpy as np
from typing import Dict
import wandb
from transformers import Trainer,PreTrainedModel,TrainingArguments,PreTrainedTokenizerBase
from transformers.data.data_collator import DataCollator
from typing import Optional, Tuple
from torch.utils.data.dataloader import DataLoader
from torch.utils.data.dataset import Dataset

use_wandb = False
if use_wandb:
  wandb.login()


if use_wandb:
  wandb.init(project="rxn_better_train")

def ner_compute_metrics(predictions, label_ids):
    label_list_str = [[id2label[x] for x in seq] for seq in label_ids]
    preds_list_str = [[id2label[x] for x in seq] for seq in predictions]
    return {
        "precision": precision(label_list_str, preds_list_str),
        "recall": recall(label_list_str, preds_list_str),
        "f1": f1(label_list_str, preds_list_str),
    }

class NERTrainer(Trainer):
  def __init__(
      self,
      model: PreTrainedModel,
      args: TrainingArguments,
      tokenizer : Optional[PreTrainedTokenizerBase] = None,
      data_collator: Optional[DataCollator] = None,
      train_dataset: Optional[Dataset] = None,
      eval_dataset: Optional[Dataset] = None,
      compute_metrics=None,
      optimizers: Tuple[torch.optim.Optimizer, torch.optim.lr_scheduler.LambdaLR] = None
  ):
    super().__init__(
      model=model,
      args=args,
      tokenizer=tokenizer,
      data_collator=data_collator,
      train_dataset=train_dataset,
      eval_dataset=eval_dataset,
      compute_metrics=compute_metrics,
      optimizers=optimizers,
    )

  def evaluate(self, eval_dataset: Optional[Dataset] = None):
    return self._prediction_loop(self.get_eval_dataloader(eval_dataset))


  def predict(self, test_dataset: Dataset):
    return self._prediction_loop(self.get_test_dataloader(test_dataset))

  def _prediction_loop(self, dataloader: DataLoader):
    model = self.model
    batch_size = dataloader.batch_size

    model.eval()

    eval_losses = []
    preds_ids = []
    label_ids = []
    with torch.no_grad():
      for dataset in tqdm(dataloader):
        input_ids = dataset['input_ids']
        attention_mask = dataset['attention_mask']
        token_type_ids = dataset['token_type_ids']
        labels = dataset['labels']
        decoder_mask = dataset['decoder_mask']

        input_ids = input_ids.to(device, dtype = torch.long)
        attention_mask = attention_mask.to(device, dtype = torch.long)
        token_type_ids = token_type_ids.to(device, dtype = torch.long)
        labels = labels.to(device, dtype = torch.long)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            labels=labels
        )

        loss = outputs[0]
        eval_losses.append(loss.item())
        decode_predictions = model.decode(outputs[1], decoder_mask)
        decode_labels = model.decode_labels(labels, decoder_mask)
        preds_ids.extend(decode_predictions)
        label_ids.extend(decode_labels)
        assert len(decode_predictions) == len(decode_labels)
        batch_size, _ = labels.shape
        for i in range(batch_size):
          assert len(decode_predictions[i]) == len(decode_labels[i])
    metrics = self.compute_metrics(preds_ids, label_ids)
    metrics['loss'] = np.mean(eval_losses)
    return {"predictions": preds_ids, "label_ids": label_ids, "metrics": metrics}

In [10]:
from transformers import get_linear_schedule_with_warmup, AdamW
from transformers import default_data_collator
from transformers import TrainingArguments
from evaluate import evaluator

model = BertForTagging(config)

model.to(device)

param_optimizer = list(model.named_parameters())
no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
optimizer_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 1e-4},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0},
]

num_training_steps = int(len(tokenized_datasets["train"])/batch_size * num_train_epochs)

optimizer = AdamW(optimizer_parameters, lr=1e-3)
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps = 0,
    num_training_steps = num_training_steps
)
train_args = TrainingArguments(output_dir = output_dir,
                 remove_unused_columns = False,
                 num_train_epochs = num_train_epochs,
                 save_strategy = "no")
trainer = NERTrainer(
    model=model,
    args=train_args,
    tokenizer=tokenizer,
    data_collator=default_data_collator,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    compute_metrics=ner_compute_metrics,
    optimizers=(optimizer, scheduler)
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
500,0.116100


TrainOutput(global_step=771, training_loss=0.10171711645856132, metrics={'train_runtime': 394.5124, 'train_samples_per_second': 15.622, 'train_steps_per_second': 1.954, 'total_flos': 805193235518976.0, 'train_loss': 0.10171711645856132, 'epoch': 1.0})

In [11]:
print(trainer.evaluate(eval_dataset=tokenized_datasets["validation"])["metrics"])

print("------------test-------------")
print(trainer.evaluate(eval_dataset=tokenized_datasets["test"])["metrics"])
# validate_model(model, eval_dataloader)

100%|██████████| 88/88 [00:15<00:00,  5.67it/s]
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'loss': 0.03717711055651307}
------------test-------------


100%|██████████| 91/91 [00:16<00:00,  5.57it/s]


{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'loss': 0.04231499269563746}
